Test the `dgaf` api on itself, it infers project information that can be used as an example test for project with modules in a named directory. it works with flit.

In [1]:
    def test_dgaf_project():
        import dgaf
        self = project = dgaf.Project()
        assert project.get_name() == "dgaf"
        requires = ["nox", "typer"]
        assert project.get_requires() == ["nox", "typer"]
        assert project.get_url()